### Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)
  

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [60]:
train_dataset=train_dataset[0:200000,0:28,0:28]
valid_dataset=valid_dataset[0:10000,0:28,0:28]
test_dataset=test_dataset[0:10000,0:28,0:28]

Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)


Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [63]:
layer1 = 1024
batch_size = 128
beta=0.001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    A1 = tf.Variable(tf.truncated_normal([image_size * image_size, layer1]))
    b1 = tf.Variable(tf.zeros([layer1]))
       
    A2 = tf.Variable(tf.truncated_normal([layer1, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))

    # Training computation. 
    logits_layer1=tf.nn.relu(tf.matmul(tf_train_dataset, A1) + b1)
    logits = tf.matmul(logits_layer1, A2) + b2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
        )+beta*(tf.nn.l2_loss(A1)+tf.nn.l2_loss(b1)+tf.nn.l2_loss(A2)+tf.nn.l2_loss(b2))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    valid_layer1=tf.nn.relu(tf.matmul(tf_valid_dataset, A1) + b1)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_layer1, A2) + b2)
    
    test_layer1=tf.nn.relu(tf.matmul(tf_test_dataset, A1) + b1)
    test_prediction = tf.nn.softmax(tf.matmul(test_layer1, A2) + b2)

In [14]:
num_steps = 3001

with tf.Session(graph=graph) as session:   
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 503.487976
Minibatch accuracy: 7.8%
Validation accuracy: 19.5%
Minibatch loss at step 500: 126.165054
Minibatch accuracy: 84.4%
Validation accuracy: 81.3%
Minibatch loss at step 1000: 96.178558
Minibatch accuracy: 79.7%
Validation accuracy: 82.2%
Minibatch loss at step 1500: 74.373634
Minibatch accuracy: 82.8%
Validation accuracy: 81.5%
Minibatch loss at step 2000: 56.372879
Minibatch accuracy: 82.0%
Validation accuracy: 83.0%
Minibatch loss at step 2500: 43.923721
Minibatch accuracy: 84.4%
Validation accuracy: 84.1%
Minibatch loss at step 3000: 33.989475
Minibatch accuracy: 85.9%
Validation accuracy: 85.2%
Test accuracy: 90.8%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [8]:
layer1 = 1024
batch_size = 128
beta=0.001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    A1 = tf.Variable(tf.truncated_normal([image_size * image_size, layer1]))
    b1 = tf.Variable(tf.zeros([layer1]))
       
    A2 = tf.Variable(tf.truncated_normal([layer1, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))

    # Training computation. 
    logits_layer1=tf.nn.relu(tf.matmul(tf_train_dataset, A1) + b1)
    logits = tf.matmul(logits_layer1, A2) + b2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
        )+beta*(tf.nn.l2_loss(A1)+tf.nn.l2_loss(b1)+tf.nn.l2_loss(A2)+tf.nn.l2_loss(b2))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    valid_layer1=tf.nn.relu(tf.matmul(tf_valid_dataset, A1) + b1)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_layer1, A2) + b2)
    
    test_layer1=tf.nn.relu(tf.matmul(tf_test_dataset, A1) + b1)
    test_prediction = tf.nn.softmax(tf.matmul(test_layer1, A2) + b2)

In [9]:
num_steps = 3001

with tf.Session(graph=graph) as session:   
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (2 * batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 641.151245
Minibatch accuracy: 13.3%
Validation accuracy: 34.8%
Minibatch loss at step 500: 191.293259
Minibatch accuracy: 100.0%
Validation accuracy: 68.4%
Minibatch loss at step 1000: 116.010979
Minibatch accuracy: 100.0%
Validation accuracy: 68.4%
Minibatch loss at step 1500: 70.355354
Minibatch accuracy: 100.0%
Validation accuracy: 68.4%
Minibatch loss at step 2000: 42.667244
Minibatch accuracy: 100.0%
Validation accuracy: 68.4%
Minibatch loss at step 2500: 25.875834
Minibatch accuracy: 100.0%
Validation accuracy: 68.4%
Minibatch loss at step 3000: 15.692508
Minibatch accuracy: 100.0%
Validation accuracy: 68.4%
Test accuracy: 73.6%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [30]:
layer1 = 1024
batch_size = 128
beta=0.2

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    A1 = tf.Variable(tf.truncated_normal([image_size * image_size, layer1]))
    b1 = tf.Variable(tf.zeros([layer1]))
       
    A2 = tf.Variable(tf.truncated_normal([layer1, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))

    # Training computation. 
    keep_prob = tf.placeholder(tf.float32)
    logits_layer1=tf.nn.relu(tf.matmul(tf_train_dataset, A1) + b1)
    logits_layer1_drop=tf.nn.dropout(logits_layer1, 0.3)
    
    logits_drop = tf.matmul(logits_layer1_drop, A2) + b2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_drop)
        )+beta*(tf.nn.l2_loss(A1)+tf.nn.l2_loss(b1)+tf.nn.l2_loss(A2)+tf.nn.l2_loss(b2))
   
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits_drop)
    
    valid_layer1=tf.nn.relu(tf.matmul(tf_valid_dataset, A1) + b1)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_layer1, A2) + b2)
    
    test_layer1=tf.nn.relu(tf.matmul(tf_test_dataset, A1) + b1)
    test_prediction = tf.nn.softmax(tf.matmul(test_layer1, A2) + b2)

In [25]:
num_steps = 3001

with tf.Session(graph=graph) as session:   
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    offset = np.random.choice(np.arange(15))
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 63256.507812
Minibatch accuracy: 16.4%
Validation accuracy: 19.9%
Minibatch loss at step 500: 1.602032
Minibatch accuracy: 85.2%
Validation accuracy: 65.5%
Minibatch loss at step 1000: 1.576485
Minibatch accuracy: 88.3%
Validation accuracy: 64.5%
Minibatch loss at step 1500: 1.606606
Minibatch accuracy: 88.3%
Validation accuracy: 66.2%
Minibatch loss at step 2000: 1.600215
Minibatch accuracy: 89.1%
Validation accuracy: 63.2%
Minibatch loss at step 2500: 1.589499
Minibatch accuracy: 86.7%
Validation accuracy: 63.7%
Minibatch loss at step 3000: 1.642288
Minibatch accuracy: 85.9%
Validation accuracy: 63.9%
Test accuracy: 68.9%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [36]:
layer1 = 512
layer2 = 256
batch_size = 128
beta=0.001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    A1 = tf.Variable(tf.truncated_normal([image_size * image_size, layer1]))
    b1 = tf.Variable(tf.zeros([layer1]))
       
    A2 = tf.Variable(tf.truncated_normal([layer1, layer2]))
    b2 = tf.Variable(tf.zeros([layer2]))
    
    A3 = tf.Variable(tf.truncated_normal([layer2, num_labels]))
    b3 = tf.Variable(tf.zeros([num_labels]))

    # Training computation. 
    logits_layer1=tf.nn.relu(tf.matmul(tf_train_dataset, A1) + b1)
    logits_layer2=tf.nn.relu(tf.matmul(logits_layer1,    A2) + b2)
    logits_layer2_drop=tf.nn.dropout(logits_layer2, 0.5)
    
    logits = tf.matmul(logits_layer2_drop, A3) + b3
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
        )+beta*(tf.nn.l2_loss(A1)+tf.nn.l2_loss(b1)+tf.nn.l2_loss(A2)+tf.nn.l2_loss(b2)+tf.nn.l2_loss(A3)+tf.nn.l2_loss(b3))

    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learnr = tf.placeholder("float")
    learning_rate = tf.train.exponential_decay(learnr, global_step, 100000, 0.95, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    valid_layer1=tf.nn.relu(tf.matmul(tf_valid_dataset, A1) + b1)
    valid_layer2=tf.nn.relu(tf.matmul(valid_layer1, A2) + b2)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_layer2, A3) + b3)
    
    test_layer1=tf.nn.relu(tf.matmul(tf_test_dataset, A1) + b1)
    test_layer2=tf.nn.relu(tf.matmul(test_layer1, A2) + b2)
    test_prediction = tf.nn.softmax(tf.matmul(test_layer2, A3) + b3)

In [43]:
batch_size = 128
beta = 0.001

hidden_nodes1 = 1024
hidden_nodes2 = 512

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # new hidden layer 1
    
    A1 = tf.Variable( tf.truncated_normal([image_size * image_size, hidden_nodes1]) )
    b1 = tf.Variable( tf.zeros([hidden_nodes1]))
    
    A2 = tf.Variable( tf.truncated_normal([hidden_nodes1, hidden_nodes2]) )
    b2 = tf.Variable( tf.zeros([hidden_nodes2]))
    
    A3 = tf.Variable( tf.truncated_normal([hidden_nodes2, num_labels])) 
    b3 = tf.Variable(tf.zeros([num_labels]))
    hidden_layer = tf.nn.relu( tf.matmul( tf_train_dataset, A1) + b1)
    
    # add dropout on hidden layer
    keep_prob = 0.5
    hidden_layer_drop = tf.nn.dropout(hidden_layer, keep_prob)
    

  
    hidden_layer2 = tf.nn.relu( tf.matmul( hidden_layer_drop, A2) + b2)

    hidden_layer_drop2 = tf.nn.dropout(hidden_layer2, keep_prob)
    logits = tf.matmul(hidden_layer_drop2, A3) + b3
    
    loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels) )
    loss = tf.reduce_mean( loss + beta * tf.nn.l2_loss(A3) )

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)    
    
    valid_relu1 = tf.nn.relu(  tf.matmul(tf_valid_dataset, A1) + b1)    
    valid_relu2 = tf.nn.relu(  tf.matmul(valid_relu1, A2) + b2)    
    
    valid_prediction = tf.nn.softmax( tf.matmul(valid_relu2, A3) + b3) 
    
    test_relu1 = tf.nn.relu( tf.matmul( tf_test_dataset, A1) + b1)
    test_relu2 = tf.nn.relu( tf.matmul( test_relu1, A2) + b2)   
    
    test_prediction = tf.nn.softmax(tf.matmul(test_relu2, A3) + b3)

In [52]:
num_steps = 3001

with tf.Session(graph=graph) as session:   
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    #offset = np.random.choice(np.arange(15))
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 9753.679688
Minibatch accuracy: 19.5%
Validation accuracy: 13.4%
Minibatch loss at step 500: nan
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Minibatch loss at step 1000: nan
Minibatch accuracy: 7.0%
Validation accuracy: 10.0%
Minibatch loss at step 1500: nan
Minibatch accuracy: 8.6%
Validation accuracy: 10.0%
Minibatch loss at step 2000: nan
Minibatch accuracy: 14.1%
Validation accuracy: 10.0%
Minibatch loss at step 2500: nan
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 3000: nan
Minibatch accuracy: 13.3%
Validation accuracy: 10.0%
Test accuracy: 10.0%


In [51]:
batch_size = 128
beta = 0.001

hidden_nodes1 = 1024
hidden_nodes2 = 512

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # new hidden layer 1
    hidden_weights = tf.Variable( tf.truncated_normal([image_size * image_size, hidden_nodes1]) )
    hidden_biases = tf.Variable( tf.zeros([hidden_nodes1]))
    hidden_layer = tf.nn.relu( tf.matmul( tf_train_dataset, hidden_weights) + hidden_biases)
    
    # add dropout on hidden layer
    keep_prob = 0.5
    hidden_layer_drop = tf.nn.dropout(hidden_layer, keep_prob)
    
    # new hidden layer 2
    hidden_weights2 = tf.Variable( tf.truncated_normal([hidden_nodes1, hidden_nodes2]) )
    hidden_biases2 = tf.Variable( tf.zeros([hidden_nodes2]))
    hidden_layer2 = tf.nn.relu( tf.matmul( hidden_layer_drop, hidden_weights2) + hidden_biases2)
    
    # add dropout on hidden layer
    hidden_layer_drop2 = tf.nn.dropout(hidden_layer2, keep_prob)
    
    # Variables.
    weights = tf.Variable( tf.truncated_normal([hidden_nodes2, num_labels])) 
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(hidden_layer_drop2, weights) + biases
    
    loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels) )
    loss = tf.reduce_mean( loss + beta * tf.nn.l2_loss(weights) )

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)    
    
    valid_relu1 = tf.nn.relu(  tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)    
    valid_relu2 = tf.nn.relu(  tf.matmul(valid_relu1, hidden_weights2) + hidden_biases2)    
    
    valid_prediction = tf.nn.softmax( tf.matmul(valid_relu2, weights) + biases) 
    
    test_relu1 = tf.nn.relu( tf.matmul( tf_test_dataset, hidden_weights) + hidden_biases)
    test_relu2 = tf.nn.relu( tf.matmul( test_relu1, hidden_weights2) + hidden_biases2)   
    
    test_prediction = tf.nn.softmax(tf.matmul(test_relu2, weights) + biases)

In [50]:
num_steps = 3001
scores = {}
for kp in [0.5, 0.6, 0.7, 0.9, 0.9, 1.0]:
    for lr in np.arange(0.0001, 0.001, 0.0001).tolist():
      #  print ">> with keep prob of " + str(kp)
       # print ">> with lr  " + str(lr)
        with tf.Session(graph=graph) as session:
          tf.initialize_all_variables().run()
          print("Initialized")
          for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : kp, learnr : lr}
            _, l, predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict )
            if (step % 500 == 0):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
          acc = accuracy(test_prediction.eval(), test_labels)
          scores[(kp, lr)] = acc
          print("Test accuracy: %.1f%%" % acc)


Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized


TypeError: Cannot interpret feed_dict key as Tensor: Tensor Tensor("Placeholder_3:0", dtype=float32) is not an element of this graph.